## 乗算レイヤ

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backword(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

In [3]:
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [4]:
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

In [5]:
price

220.00000000000003

In [6]:
dprice = 1

In [7]:
dapple_price, dtax = mul_tax_layer.backword(dprice)

In [8]:
dapple, dapple_num = mul_apple_layer.backword(dapple_price)

In [9]:
dapple, dapple_num, dtax

(2.2, 110.00000000000001, 200)

## 加算レイヤ

In [10]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [11]:
apple = 100
apple_num = 2 
orange = 150
orange_num = 3
tax = 1.1

In [12]:
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [13]:
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

In [14]:
dprice = 1
dall_price, dtax = mul_tax_layer.backword(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backword(dorange_price)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)

In [15]:
price

715.0000000000001

In [16]:
dapple_num, dapple, dorange, dorange_num, dtax

(110.00000000000001, 2.2, 3.3000000000000003, 165.0, 650)

## ReLUレイヤ

In [17]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [18]:
relu_layer = Relu()

In [19]:
xs = np.array([[1.0, -2.0], [-3.0, 5.0]])
ys = np.array([[0.1, 0.2], [-0.3, 0.4]])

In [20]:
relu_layer.forward(xs)

array([[1., 0.],
       [0., 5.]])

In [21]:
relu_layer.backward(ys)

array([[0.1, 0. ],
       [0. , 0.4]])

## Sigmoidレイヤ

In [22]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [23]:
xs = np.array([1.0, 2.0, 100])

In [24]:
sigmoid_layer = Sigmoid()

In [25]:
sigmoid_layer.forward(xs)

array([0.73105858, 0.88079708, 1.        ])

In [26]:
sigmoid_layer.backward(xs)

array([0.19661193, 0.20998717, 0.        ])

## Affine/Softmax レイヤ

In [27]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [28]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [29]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return - np.sum(t * np.log(y + delta))

In [30]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
       
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
        
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx